In [1]:
!pip install ucimlrepo

In [90]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from ucimlrepo import fetch_ucirepo 
import plotly
import plotly.graph_objs as go
import plotly.express as px
from plotly.subplots import make_subplots

  
# fetch dataset 
credit_approval = fetch_ucirepo(id=27) 
  
# data (as pandas dataframes) 
X = credit_approval.data.features 
y = credit_approval.data.targets 
  
# metadata 
# print(credit_approval.metadata) 
  


#создаем датафрейм в удобном для нас формате pandas
df = pd.DataFrame(data=credit_approval.data.features, columns=credit_approval.variables.name)
#столбец A16 - таргет ("выдан займ или нет")
df['A16'] = y
#выведем пять первых строк нашего датафрейма, чтобы примерно понимать, как выглядят наши данные

print(df.head())




name A16  A15    A14 A13 A12  A11 A10 A9    A8 A7 A6 A5 A4     A3     A2 A1
0      +    0  202.0   g   f    1   t  t  1.25  v  w  g  u  0.000  30.83  b
1      +  560   43.0   g   f    6   t  t  3.04  h  q  g  u  4.460  58.67  a
2      +  824  280.0   g   f    0   f  t  1.50  h  q  g  u  0.500  24.50  a
3      +    3  100.0   g   t    5   t  t  3.75  v  w  g  u  1.540  27.83  b
4      +    0  120.0   s   f    0   f  t  1.71  v  w  g  u  5.625  20.17  b


In [91]:
# variable information 
print(credit_approval.variables) 
#Заметим, что категориальные переменные по определению могут принимать только ограниченное и обычно фиксированное количество возможных значений. Это свойство понадобится нам 
#при заполнении пропусков
#Так же  категориальные переменные в данном датафрейме  принимают только буквенные значения
#для удобства разделим переменные на категориальные и непрерывные
continuous = credit_approval.variables[credit_approval.variables['type']=='Continuous']
categorical = credit_approval.variables[credit_approval.variables['type']=='Categorical']


   name     role         type demographic description units missing_values
0   A16   Target  Categorical        None        None  None             no
1   A15  Feature   Continuous        None        None  None             no
2   A14  Feature   Continuous        None        None  None            yes
3   A13  Feature  Categorical        None        None  None             no
4   A12  Feature  Categorical        None        None  None             no
5   A11  Feature   Continuous        None        None  None             no
6   A10  Feature  Categorical        None        None  None             no
7    A9  Feature  Categorical        None        None  None             no
8    A8  Feature   Continuous        None        None  None             no
9    A7  Feature  Categorical        None        None  None            yes
10   A6  Feature  Categorical        None        None  None            yes
11   A5  Feature  Categorical        None        None  None            yes
12   A4  Feature  Categor

In [92]:
continuous

,name,role,type,demographic,description,units,missing_values
1,A15,Feature,Continuous,None,None,None,no
2,A14,Feature,Continuous,None,None,None,yes
5,A11,Feature,Continuous,None,None,None,no
8,A8,Feature,Continuous,None,None,None,no
13,A3,Feature,Continuous,None,None,None,no
14,A2,Feature,Continuous,None,None,None,yes


In [93]:
categorical

,name,role,type,demographic,description,units,missing_values
0,A16,Target,Categorical,None,None,None,no
3,A13,Feature,Categorical,None,None,None,no
4,A12,Feature,Categorical,None,None,None,no
6,A10,Feature,Categorical,None,None,None,no
7,A9,Feature,Categorical,None,None,None,no
9,A7,Feature,Categorical,None,None,None,yes
10,A6,Feature,Categorical,None,None,None,yes
11,A5,Feature,Categorical,None,None,None,yes
12,A4,Feature,Categorical,None,None,None,yes
15,A1,Feature,Categorical,None,None,None,yes


In [95]:
#Заполним пропуски, а так же проиллюстрируем их заполнение:
#для начала в таблицу started_blanks запишем количество пропусков в каждом столбце
started_blanks = df.isna().sum()
#непосредственно заполним пропуска
for name in credit_approval.variables.name:
    if (df[name].isna().sum()!=0):
        if ((df[name].dtype == int) or (df[name].dtype == float)): #если в столбце не строки, а числа, то заполним средним значением (если переменные непрерывные)
            fill_value =df[name].mean()
            df[name].fillna(fill_value, inplace=True)
        else:
            #если переменная категориальная, то по свойству, описанному выше, кажется логичным заполнить пропуски самым поплярным значением в столбце
            counted_values = df[name].value_counts()
            most_common =counted_values.idxmax()
            df[name].fillna(most_common, inplace=True)
#создадим еще одну таблицу с количеством пропусков, но уже в заполненном датафрейме
finished_blanks = df.isna().sum()

print("До заполнения")
print(started_blanks)
print("После заполнения")
print(finished_blanks)

До заполнения
name
A16     0
A15     0
A14    13
A13     0
A12     0
A11     0
A10     0
A9      0
A8      0
A7      9
A6      9
A5      6
A4      6
A3      0
A2     12
A1     12
dtype: int64
После заполнения
name
A16    0
A15    0
A14    0
A13    0
A12    0
A11    0
A10    0
A9     0
A8     0
A7     0
A6     0
A5     0
A4     0
A3     0
A2     0
A1     0
dtype: int64


Теперь изучим распределение переменных в столбцах, где каждый элемент - числовое значение. Иными словами, рассмотрим распределение непрерывных переменных:

Для начала выведем основную информацию о столбцах с непрерывными переменными

In [96]:
df.describe()
#count - The number of not-empty values.
# mean - The average (mean) value.
# std - The standard deviation.
# min - the minimum value.
# 25% - The 25% percentile*.
# 50% - The 50% percentile*.
# 75% - The 75% percentile*.
# max - the maximum value.

name,A15,A14,A11,A8,A3,A2
count,690.000000,690.000000,690.00000,690.000000,690.000000,690.000000
mean,1017.385507,184.014771,2.40000,2.223406,4.758725,31.568171
std,5210.102598,172.159274,4.86294,3.346513,4.978163,11.853273
min,0.000000,0.000000,0.00000,0.000000,0.000000,13.750000
25%,0.000000,80.000000,0.00000,0.165000,1.000000,22.670000
50%,5.000000,160.000000,0.00000,1.000000,2.750000,28.625000
75%,395.500000,272.000000,3.00000,2.625000,7.207500,37.707500
max,100000.000000,2000.000000,67.00000,28.500000,28.000000,80.250000


In [97]:

updatemenu = [dict()]
buttons = []
fig = go.Figure()
for i in continuous.name:
    fig.add_trace(px.histogram(df, x=i).data[0])

for i, col in enumerate(continuous.name):

    visibles = [False] * len(continuous.name)
    visibles[i] = True
    buttons.append(dict(method='restyle',
                        label=col,
                        args=["visible", visibles]
                        ))

updatemenu[0]['buttons'] = buttons
updatemenu[0]['direction'] = 'down'
updatemenu[0]['showactive'] = True

fig.update_layout(showlegend=True, updatemenus=updatemenu)
fig.update_layout(
     title={
        "text": "Распределение непрерывных переменных",
        "x": 0.5
    },
    xaxis_title="Диапазон",
    yaxis_title="Количество значений"
)
fig.show()

Теперь рассмотрим распределение категориальных переменных

In [98]:
updatemenu = [dict()]
buttons = []
fig2 = go.Figure()

for i in categorical.name:
    fig2.add_trace(px.histogram(df, x=i).data[0])

for i, col in enumerate(categorical.name):

    visibles = [False] * len(categorical.name)
    visibles[i] = True
    buttons.append(dict(method='restyle',
                        label=col,
                        args=["visible", visibles]
                        ))

updatemenu[0]['buttons'] = buttons
updatemenu[0]['direction'] = 'down'
updatemenu[0]['showactive'] = True

fig2.update_layout(showlegend=True, updatemenus=updatemenu)
fig2.update_layout(
     title={
        "text": "Распределение непрерывных переменных",
        "x": 0.5
    },
    xaxis_title="Значение",
    yaxis_title="Количество значений"
)
fig2.show()

Также необходимо посмотреть на корреляцию непрерывных переменных

In [99]:
df.corr()

name,A15,A14,A11,A8,A3,A2
name,,,,,,
A15,1.000000,0.065609,0.063692,0.051345,0.123121,0.018539
A14,0.065609,1.000000,-0.119809,-0.076389,-0.222346,-0.077161
A11,0.063692,-0.119809,1.000000,0.322330,0.271207,0.185575
A8,0.051345,-0.076389,0.322330,1.000000,0.298902,0.392787
A3,0.123121,-0.222346,0.271207,0.298902,1.000000,0.201316
A2,0.018539,-0.077161,0.185575,0.392787,0.201316,1.000000


In [100]:
fig_cor = px.imshow(df.corr(), text_auto=True)
fig_cor.update_layout(
     title={
        "text": "Корреляция непрерывных переменных",
        "x": 0.5
    },

)
fig_cor.show()